# [modified unsuccessfully] Dynamic Section Retrieval with LlamaParse
### (with OpenAI's gpt-4o, replaced by an LLM on Groq and an open embedding model on Hugging Face

This notebook showcases a concept called "dynamic section retrieval".

A common problem with naive RAG approaches is that each document is hierarchically organized by section, but standard chunking/retrieval searches for chunks that can be fragments of the entire section and miss out on relevant context.

Dynamic section retrieval takes into account entire contiguous sections as metadata during retrieval, avoiding the problem of retrieving section fragments.
1. First, tag chunks of a long document with the sections they correspond to, through structured extraction.
2. Do two-pass retrieval. After initial semantic search, dynamically pull in the entire section through metadata filtering.

![](dynamic_section_retrieval_img.png)

This helps provide a solution to the common chunking problem of retrieving chunks that are only subsets of the entire section you're meant to retrieve.

In [1]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

#%ls -la /content/drive/MyDrive/docs4Parsing/

## Setup

Install core packages and download relevant files. Here we load some popular ICLR 2024 papers.

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
!pip install llama-index
!pip install llama-index-core
!pip install llama-parse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.1/613.1 kB 43.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenac

In [5]:

file_path = "https://raw.githubusercontent.com/tlyim/DynSecRetrieval/main/docs4Parsing/"

papers = [
    "accounting.pdf",
    "selfrag.pdf",
#    "deloitte_pp35-39.pdf",
#    "next_pp39-52.pdf",
]

urls = [file_path + filename for filename in papers]

#data_dir = "/content/drive/MyDrive/docs4Parsing"
data_dir = "docs4Parsing"


In [6]:
!mkdir "{data_dir}"
for url, paper in zip(urls, papers):
    !wget "{url}" -O "{data_dir}/{paper}"

#papers
#urls


mkdir: cannot create directory ‘docs4Parsing’: File exists
--2024-11-23 15:21:49--  https://raw.githubusercontent.com/tlyim/DynSecRetrieval/main/docs4Parsing/accounting.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 246063 (240K) [application/octet-stream]
Saving to: ‘docs4Parsing/accounting.pdf’

docs4Parsing/accoun 100%[===================>] 240.30K  --.-KB/s    in 0.001s  

2024-11-23 15:21:49 (174 MB/s) - ‘docs4Parsing/accounting.pdf’ saved [246063/246063]

--2024-11-23 15:21:49--  https://raw.githubusercontent.com/tlyim/DynSecRetrieval/main/docs4Parsing/selfrag.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.c

In [7]:
# Show the downloaded files

import subprocess
result = subprocess.run(['ls', '-la', data_dir], capture_output=True, text=True, check=True)
print(result.stdout)  # Print the output


total 1180
drwxrwxrwx+ 2 codespace root   4096 Nov 23 15:19 .
drwxrwxrwx+ 4 codespace root   4096 Nov 23 15:19 ..
-rw-rw-rw-  1 codespace root 246063 Nov 23 15:21 accounting.pdf
-rw-rw-rw-  1 codespace root 308160 Nov 23 15:19 deloitte_pp35-39.pdf
-rw-rw-rw-  1 codespace root  41562 Nov 23 15:19 next_pp39-52.pdf
-rw-rw-rw-  1 codespace root 590111 Nov 23 15:21 selfrag.pdf



In [8]:
!pip install llama_index.llms.groq
!pip install llama_index.embeddings.huggingface


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
import os

# Access the secrets from environment variables
os.environ["LLAMA_CLOUD_API_KEY"] = os.getenv('LLAMA_CLOUD_API_KEY')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')
os.environ["HF_TOKEN"] = os.getenv('HF_TOKEN')   # HUGGING FACE

# If running this on Google Colab, you can use the following code to access the secrets
#from google.colab import userdata
#os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get('LLAMA_CLOUD_API_KEY')
#os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
#os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')   # HUGGING FACE

# Display environment variables (for debugging purposes)
#print(os.environ.get('LLAMA_CLOUD_API_KEY'))


In [9]:
# Get API key interactively from user input

#import getpass

#os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass()
#os.environ["GROQ_API_KEY"] = getpass.getpass()
#os.environ["HF_TOKEN"] = getpass.getpass()

#### Define LLM and Embedding Model

In [13]:
from llama_index.llms.groq import Groq
#llm = Groq(model="llama3-8b-8192")
#llm = Groq(model="llama-3.2-11b-vision-preview")
#llm = Groq(model="llama3-70b-8192")
llm = Groq(model="llama-3.2-90b-vision-preview")

# Instead of OpenAIEmbedding, use a compatible embedding model (like HuggingFaceEmbedding)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="mixedbread-ai/mxbai-embed-large-v1")

#from llama_index.llms.openai import OpenAI
#llm = OpenAI(model="gpt-4o")
#from llama_index.embeddings.openai import OpenAIEmbedding
#embed_model = OpenAIEmbedding(model="text-embedding-3-large")

from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model


## One common interface for all LLMs

You can import several LLMs on LlamaIndex. LlamaIndex exposes a common interface to all LLM integrations.

In [11]:
# Testing
#response = llm.complete("Tell me a joke about a rabbit")
#print(response)

#### Parse Documents with **LlamaParse**

In [ ]:
from llama_parse import LlamaParse

parser = LlamaParse(#api_key="llx-...", # put your LlamaParse api key here
                    premium_mode=True,
                    result_type="markdown")

In [52]:
from pathlib import Path

paper_dicts = {}

for paper_path in papers:
    paper_base = Path(paper_path).stem
    full_paper_path = str(Path(data_dir) / paper_path)
    md_json_objs = parser.get_json_result(full_paper_path)
    json_dicts = md_json_objs[0]["pages"]
    paper_dicts[paper_path] = {
        "paper_path": full_paper_path,
        "json_dicts": json_dicts,
    }

Started parsing the file under job_id fc58bbb0-cb47-4ccb-b8bd-6704e7cc7ac2
Started parsing the file under job_id 4ab8cd5c-c33b-43e9-9f02-d76e555bc953


#### Get Text Nodes

Convert the dictionary above into TextNode objects that we can put into a vector store.

In [53]:
from llama_index.core.schema import TextNode
from typing import Optional

In [54]:
# NOTE: these are utility functions to sort the dumped images by the page number
# (they are formatted like "{uuid}-{page_num}.jpg"
import re


def get_page_number(file_name):
    match = re.search(r"-page-(\d+)\.jpg$", str(file_name))
    if match:
        return int(match.group(1))
    return 0


def _get_sorted_image_files(image_dir):
    """Get image files sorted by page."""
    raw_files = [f for f in list(Path(image_dir).iterdir()) if f.is_file()]
    sorted_files = sorted(raw_files, key=get_page_number)
    return sorted_files

In [55]:
from copy import deepcopy
from pathlib import Path


# attach image metadata to the text nodes
def get_text_nodes(json_dicts, paper_path):
    """Split docs into nodes, by separator."""
    nodes = []

    md_texts = [d["md"] for d in json_dicts]

    for idx, md_text in enumerate(md_texts):
        chunk_metadata = {
            "page_num": idx + 1,
            "paper_path": paper_path,
        }
        node = TextNode(
            text=md_text,
            metadata=chunk_metadata,
        )
        nodes.append(node)

    return nodes

In [56]:
# this will combine all nodes from all papers into a single list
all_text_nodes = []
text_nodes_dict = {}
for paper_path, paper_dict in paper_dicts.items():
    json_dicts = paper_dict["json_dicts"]
    text_nodes = get_text_nodes(json_dicts, paper_dict["paper_path"])
    all_text_nodes.extend(text_nodes)
    text_nodes_dict[paper_path] = text_nodes

#### **AddedByMe:**

Checking the structure of the text_nodes_dict dictionary indexed by the pdf filename


In [57]:
#print(all_text_nodes)
#print(text_nodes_dict)

#type(text_nodes_dict)

# Number of keys
num_keys = len(text_nodes_dict)
print(f"Number of keys: {num_keys}")

# Lengths of vectors
for key, value in text_nodes_dict.items():
    print(f"Length of vector for key '{key}': {len(value)}")

# Print the strings in each node of a document row by row
for key, value in text_nodes_dict.items():
    print(f"{key}:")
    for item in value:
        print(f"  {item}")



Number of keys: 2
Length of vector for key 'accounting.pdf': 2
Length of vector for key 'selfrag.pdf': 2
accounting.pdf:
  Node ID: 58f9fd20-509b-4e43-9fbc-5a300eef6663
Text: # Numerical Accounting in the Shuffle Model of Differential
Privacy  Antti Koskela  antti.h.koskela@nokia-bell-labs.com  Nokia
Bell Labs  University of Helsinki  Mikko Heikkilä
mikko.a.heikkila@helsinki.fi  Department of Computer Science
University of Helsinki  Antti Honkela  antti.honkela@helsinki.fi
Department of Computer Science  Universit...
  Node ID: 9d058015-6edf-44fb-bb7a-c4290a5f586d
Text: Published in Transactions on Machine Learning Research (MM/YYYY)
thus obtained bounds can be up to orders of magnitudes tighter than
the existing bounds from the literature. We also evaluate how
significantly adversaries with varying capabilities differ in terms of
the resulting privacy bounds using the k-randomised response
mechanism. For conc...
selfrag.pdf:
  Node ID: 2199d2db-6e2e-4fed-8e64-b2e5468c4885
Text: # SELF

## Add Section Metadata

The first step is to extract out a map of all sections from the text of each document. We create a workflow that extracts out if a section heading exists on each page, and merges it together into a combined list. We then run a reflection step to review/correct the extracted sections to make sure everything is correct.

Once we have a map of all the sections and the page numbers they start at, we can add the appropriate section ID as metadata to each chunk.

#### Define Section Schema to Extract Into

Here we define the output schema which allows us to extract out the section metadata from each section of the document. This will give us a full table of contents of each section.

In [58]:
from pydantic import BaseModel, Field
from typing import List, Optional


class SectionOutput(BaseModel):
    """The metadata for a given section. Includes the section name, title, page that it starts on, and more."""

    section_name: str = Field(
        ..., description="The current section number (e.g. section_name='3.2')"
    )
    section_title: str = Field(
        ...,
        description="The current section title associated with the number (e.g. section_title='Experimental Results')",
    )

    start_page_number: int = Field(..., description="The start page number.")
    is_subsection: bool = Field(
        ...,
        description="True if it's a subsection (e.g. Section 3.2). False if it's not a subsection (e.g. Section 3)",
    )
    description: Optional[str] = Field(
        None,
        description="The extracted line from the source text that indicates this is a relevant section.",
    )

    def get_section_id(self):
        """Get section id."""
        return f"{self.section_name}: {self.section_title}"


class SectionsOutput(BaseModel):
    """A list of all sections."""

    sections: List[SectionOutput]


class ValidSections(BaseModel):
    """A list of indexes, each corresponding to a valid section."""

    valid_indexes: List[int] = Field(
        "List of valid section indexes. Do NOT include sections to remove."
    )

#### Extract into Section Outputs

Use LlamaIndex structured output capabilities to iterate through each page and extract out relevant section metadata. Note: some pages may contain no section metadata (there are no sections that begin on that page).

In [59]:
from llama_index.llms.groq import Groq
from llama_index.llms.openai import OpenAI

from llama_index.core.prompts import ChatPromptTemplate, ChatMessage
from llama_index.core.llms import LLM
from llama_index.core.async_utils import run_jobs, asyncio_run
import json


async def aget_sections(
    doc_text: str, llm: Optional[LLM] = None
) -> List[SectionOutput]:
    """Get extracted sections from a provided text."""

    system_prompt = """\
    You are an AI document assistant tasked with extracting out section metadata from a document text.

- You should ONLY extract out metadata if the document text contains the beginning of a section.
- The metadata schema is listed below - you should extract out the section_name, section_title, start page number, description.
- A valid section MUST begin with a hashtag (#) and have a number (e.g. "1 Introduction" or "Section 1 Introduction"). \
Note: Not all hashtag (#) lines are valid sections.

- You can extract out multiple section metadata if there are multiple sections on the page.
- If there are no sections that begin in this document text, do NOT extract out any sections.
- A valid section MUST be clearly delineated in the document text. Do NOT extract out a section if it is mentioned, \
but is not actually the start of a section in the document text.
- A Figure or Table does NOT count as a section.

    The user will give the document text below.

    """
    #llm = llm #or OpenAI(model="gpt-4o")
    #llm = Groq(model="llama3-8b-8192")
    #llm = Groq(model="llama-3.2-11b-vision-preview")
    #llm = Groq(model="llama3-70b-8192")
    llm = Groq(model="llama-3.2-90b-vision-preview")

    chat_template = ChatPromptTemplate(
        [
            ChatMessage.from_str(system_prompt, "system"),
            ChatMessage.from_str("Document text: {doc_text}", "user"),
        ]
    )
    result = await llm.astructured_predict(
        SectionsOutput, chat_template, doc_text=doc_text
    )
    return result.sections


async def arefine_sections(
    sections: List[SectionOutput], llm: Optional[LLM] = None
) -> List[SectionOutput]:
    """Refine sections based on extracted text."""

    system_prompt = """\
    You are an AI review assistant tasked with reviewing and correcting another agent's work in extracting sections from a document.

    Below is the list of sections with indexes. The sections may be incorrect in the following manner:
    - There may be false positive sections - some sections may be wrongly extracted - you can tell by the sequential order of the rest of the sections
    - Some sections may be incorrectly marked as subsections and vice-versa
    - You can use the description which contains extracted text from the source document to see if it actually qualifies as a section.

    Given this, return the list of indexes that are valid. Do NOT include the indexes to be removed.

    """
    #llm = llm or OpenAI(model="gpt-4o")
    #llm = Groq(model="llama3-8b-8192")
    #llm = Groq(model="llama-3.2-11b-vision-preview")
    #llm = Groq(model="llama3-70b-8192")
    llm = Groq(model="llama-3.2-90b-vision-preview")


    chat_template = ChatPromptTemplate(
        [
            ChatMessage.from_str(system_prompt, "system"),
            ChatMessage.from_str("Sections in text:\n\n{sections}", "user"),
        ]
    )

    section_texts = "\n".join(
        [f"{idx}: {json.dumps(s.dict())}" for idx, s in enumerate(sections)]
    )

    result = await llm.astructured_predict(
        ValidSections, chat_template, sections=section_texts
    )
    valid_indexes = result.valid_indexes

    new_sections = [s for idx, s in enumerate(sections) if idx in valid_indexes]
    return new_sections


async def acreate_sections(text_nodes_dict):
    sections_dict = {}
    for paper_path, text_nodes in text_nodes_dict.items():
        all_sections = []

        tasks = [aget_sections(n.get_content(metadata_mode="all")) for n in text_nodes]

        async_results = await run_jobs(tasks, workers=8, show_progress=True)
        all_sections = [s for r in async_results for s in r]

        all_sections = await arefine_sections(all_sections)
        sections_dict[paper_path] = all_sections
    return sections_dict

In [64]:
sections_dict = asyncio_run(acreate_sections(text_nodes_dict))

100%|██████████| 2/2 [00:02<00:00,  1.06s/it]


AttributeError: 'str' object has no attribute 'valid_indexes'

In [ ]:
sections_dict["accounting.pdf"]

[SectionOutput(section_name='1', section_title='Introduction', start_page_number=1, is_subsection=False, description='The shuffle model of differential privacy (DP) is a distributed privacy model which sits between the high trust–high utility centralised DP, and the low trust–low utility local DP (LDP). In the shuffle model, the individual results from local randomisers are only released through a secure shuffler. This additional randomisation leads to “amplification by shuffling”, resulting in better privacy bounds against adversaries without access to the unshuffled local results.'),
 SectionOutput(section_name='1.1', section_title='Related work', start_page_number=2, is_subsection=True, description='DP was originally defined in the central model assuming a trusted aggregator by Dwork et al. (2006), while the fully distributed LDP was formally introduced and analysed by Kasiviswanathan et al. (2011).'),
 SectionOutput(section_name='2', section_title='Background: numerical privacy acc

In [ ]:
sections_dict["selfrag.pdf"]

[SectionOutput(section_name='1', section_title='INTRODUCTION', start_page_number=1, is_subsection=False, description='SELF-RAG: LEARNING TO RETRIEVE, GENERATE, AND CRITIQUE THROUGH SELF-REFLECTION')]

In [ ]:
# [Optional] SAVE
import pickle

pickle.dump(sections_dict, open("sections_dict.pkl", "wb"))

In [ ]:
# [Optional] LOAD
sections_dict = pickle.load(open("sections_dict.pkl", "rb"))

#### Annotate each chunk with the section metadata

In the section above we've extracted out a TOC of all sections/subsections and their page numbers. Given this we can just do one forward pass through all the chunks, and annotate them with the section they correspond to (e.g. the section/subsection with the highest page number less than the page number of the chunk).

In [ ]:
def annotate_chunks_with_sections(chunks, sections):
    main_sections = [s for s in sections if not s.is_subsection]
    # subsections include the main sections too (some sections have no subsections etc.)
    sub_sections = sections

    main_section_idx, sub_section_idx = 0, 0
    for idx, c in enumerate(chunks):
        cur_page = c.metadata["page_num"]
        while (
            main_section_idx + 1 < len(main_sections)
            and main_sections[main_section_idx + 1].start_page_number <= cur_page
        ):
            main_section_idx += 1
        while (
            sub_section_idx + 1 < len(sub_sections)
            and sub_sections[sub_section_idx + 1].start_page_number <= cur_page
        ):
            sub_section_idx += 1

        cur_main_section = main_sections[main_section_idx]
        cur_sub_section = sub_sections[sub_section_idx]

        c.metadata["section_id"] = cur_main_section.get_section_id()
        c.metadata["sub_section_id"] = cur_sub_section.get_section_id()

In [ ]:
for paper_path, text_nodes in text_nodes_dict.items():
    sections = sections_dict[paper_path]
    annotate_chunks_with_sections(text_nodes, sections)

You can choose to save these nodes if you'd like.

In [ ]:
# SAVE
import pickle

pickle.dump(text_nodes_dict, open("iclr_text_nodes.pkl", "wb"))

**LOAD**: If you've already saved nodes, run the below cell to load from an existing file.

In [ ]:
# LOAD
import pickle

text_nodes_dict = pickle.load(open("iclr_text_nodes.pkl", "rb"))

In [ ]:
all_text_nodes = []
for paper_path, text_nodes in text_nodes_dict.items():
    all_text_nodes.extend(text_nodes)

In [ ]:
len(all_text_nodes)

4

### Build Indexes

Once the text nodes are ready, we feed into our vector store index abstraction, which will index these nodes into a simple in-memory vector store (of course, you should definitely check out our 40+ vector store integrations!)

Besides vector indexing, we **also** store a mapping of paper path to the summary index. This allows us to perform document-level retrieval - retrieve all chunks relevant to a given document.

In [ ]:
!pip install chromadb
#!pip install llama_index.vector_stores
!pip install llama_index.vector_stores.chroma

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex

persist_dir = "storage_chroma"

vector_store = ChromaVectorStore.from_params(
    collection_name="text_nodes", persist_dir=persist_dir
)
index = VectorStoreIndex.from_vector_store(vector_store)

**NOTE**: Don't run the block below if you've already inserted the nodes. Only run if it's your first time!!

In [ ]:
index.insert_nodes(all_text_nodes)

## Setup Dynamic, Section-Level Retrieval

We now setup a retriever that will allow us to retrieve an entire contiguous section in a document, instead of a chunk of it. This is useful for preserving the entire context within a doc.

- Step 1: Do chunk-level retrieval to find the relevant chunks.
- Step 2: For each chunk, identify the section that it corresponds to.
- Step 3: Do a second retrieval pass using metadata filters to find the entire contiguous section that matches the chunk, and return that as a continguous node.
- Step 4: Feed the contiguous sections into the LLM.

In [ ]:
from llama_index.llms.openai import OpenAI

#llm = OpenAI(model="gpt-4o")
#llm = Groq(model="llama3-70b-8192")
llm = Groq(model="llama-3.2-90b-vision-preview")

In [ ]:
chunk_retriever = index.as_retriever(similarity_top_k=3)

In [ ]:
from llama_index.core.vector_stores.types import (
    VectorStoreInfo,
    VectorStoreQuerySpec,
    MetadataInfo,
    MetadataFilters,
    FilterCondition,
)
from llama_index.core.schema import NodeWithScore


def section_retrieve(query: str, verbose: bool = False) -> List[NodeWithScore]:
    """Retrieve sections."""
    if verbose:
        print(f">> Identifying the right sections to retrieve")
    chunk_nodes = chunk_retriever.retrieve(query)

    all_section_nodes = {}
    for node in chunk_nodes:
        section_id = node.node.metadata["section_id"]
        if verbose:
            print(f">> Retrieving section: {section_id}")
        filters = MetadataFilters.from_dicts(
            [
                {"key": "section_id", "value": section_id, "operator": "=="},
                {
                    "key": "paper_path",
                    "value": node.node.metadata["paper_path"],
                    "operator": "==",
                },
            ],
            condition=FilterCondition.AND,
        )

        # TODO: make node_ids not positional
        section_nodes_raw = index.vector_store.get_nodes(node_ids=None, filters=filters)
        section_nodes = [NodeWithScore(node=n) for n in section_nodes_raw]
        # order and consolidate nodes
        section_nodes_sorted = sorted(
            section_nodes, key=lambda x: x.metadata["page_num"]
        )

        all_section_nodes.update({n.id_: n for n in section_nodes_sorted})
    return all_section_nodes.values()

In [ ]:
nodes = section_retrieve(
#    "Give me a full overview of the benchmark details in SWE Bench", verbose=True
    "privacy accounting", verbose=True
)

>> Identifying the right sections to retrieve
>> Retrieving section: 1: Introduction


ValueError: Expected IDs to be a non-empty list, got 0 IDs in get.

In [ ]:
for n in nodes:
    print(n.node.metadata)

NameError: name 'nodes' is not defined

In [ ]:
nodes = section_retrieve(
    "Give me details of all additional experimental results in the Metra paper",
    verbose=True,
)

>> Identifying the right sections to retrieve
>> Retrieving section: F: ADDITIONAL RESULTS
>> Retrieving section: 5: EXPERIMENTS
>> Retrieving section: F: ADDITIONAL RESULTS


In [ ]:
for n in nodes:
    print(n.node.metadata)

{'page_num': 21, 'paper_path': 'iclr_docs/metra.pdf', 'section_id': 'F: ADDITIONAL RESULTS', 'sub_section_id': 'F.1: FULL QUALITATIVE RESULTS'}
{'page_num': 22, 'paper_path': 'iclr_docs/metra.pdf', 'section_id': 'F: ADDITIONAL RESULTS', 'sub_section_id': 'F.4: Additional Baselines'}
{'page_num': 6, 'paper_path': 'iclr_docs/metra.pdf', 'section_id': '5: EXPERIMENTS', 'sub_section_id': '5: EXPERIMENTS'}
{'page_num': 7, 'paper_path': 'iclr_docs/metra.pdf', 'section_id': '5: EXPERIMENTS', 'sub_section_id': '5.2: QUALITATIVE COMPARISON'}
{'page_num': 8, 'paper_path': 'iclr_docs/metra.pdf', 'section_id': '5: EXPERIMENTS', 'sub_section_id': '5.3: QUANTITATIVE COMPARISON'}


### Try out Section-Level Retrieval as a Full RAG Pipeline

Now that we've defined the retriever, we can plug the retrieved results into an LLM to create a full RAG pipeline!

Our response synthesizers help handle dumping context into the LLM prompt window while accounting for context window limitations.

In [ ]:
from llama_index.core.query_engine import CustomQueryEngine
from llama_index.core.response_synthesizers import TreeSummarize, BaseSynthesizer


class SectionRetrieverRAGEngine(CustomQueryEngine):
    """RAG Query Engine."""

    synthesizer: BaseSynthesizer
    verbose: bool = True

    def __init__(self, *args, **kwargs):
        super().__init__(synthesizer=TreeSummarize(llm=llm))

    def custom_query(self, query_str: str):
        nodes = section_retrieve(query_str, verbose=self.verbose)
        response_obj = self.synthesizer.synthesize(query_str, nodes)
        return response_obj

In [ ]:
query_engine = SectionRetrieverRAGEngine()

In [ ]:
response = query_engine.query(
    "Tell me more about how difficulty correlates with context length in SWEBench"
)
print(str(response))

>> Identifying the right sections to retrieve
>> Retrieving section: A: BENCHMARK DETAILS
>> Retrieving section: 5: RESULTS
>> Retrieving section: A: BENCHMARK DETAILS
In SWEBench, difficulty correlates with context length in a way that as the total context length increases, model performance tends to drop. This is observed across various models, including Claude 2, which shows a significant decrease in performance with longer context lengths. The models often struggle to localize the problematic code that needs updating when presented with a lot of code that may not be directly related to the issue at hand. This suggests that models can become distracted by additional context, which aligns with findings from other studies indicating that models may be sensitive to the relative location of target sequences. Even when increasing the maximum context size improves recall with respect to the oracle files, performance still drops, indicating that models are ineffective at localizing the nec

In [ ]:
response = query_engine.query(
    "Give me a full overview of the benchmark details in SWE Bench"
)
print(str(response))

>> Identifying the right sections to retrieve
>> Retrieving section: A: BENCHMARK DETAILS
>> Retrieving section: 2: BENCHMARK CONSTRUCTION
>> Retrieving section: A: BENCHMARK DETAILS
SWE-bench is a benchmark designed to evaluate language models in a realistic software engineering setting by using GitHub issues and pull requests from popular repositories. The benchmark involves generating a pull request that addresses a given issue and passes related tests. The construction of SWE-bench involves a three-stage pipeline:

1. **Repo Selection and Data Scraping**: Pull requests are collected from 12 popular open-source Python repositories on GitHub, resulting in approximately 90,000 PRs. These repositories are chosen for their better maintenance, clear contributor guidelines, and comprehensive test coverage.

2. **Attribute-Based Filtering**: Candidate tasks are created by selecting merged PRs that resolve a GitHub issue and contribute tests. This indicates that the user likely added tests 

In [ ]:
for n in response.source_nodes:
    print(n.metadata)

{'page_num': 15, 'paper_path': 'iclr_docs/swebench.pdf', 'section_id': 'A: BENCHMARK DETAILS', 'sub_section_id': 'A.1: HIGH LEVEL OVERVIEW'}
{'page_num': 16, 'paper_path': 'iclr_docs/swebench.pdf', 'section_id': 'A: BENCHMARK DETAILS', 'sub_section_id': 'A.2: CONSTRUCTION PROCESS'}
{'page_num': 17, 'paper_path': 'iclr_docs/swebench.pdf', 'section_id': 'A: BENCHMARK DETAILS', 'sub_section_id': 'A.2: CONSTRUCTION PROCESS'}
{'page_num': 18, 'paper_path': 'iclr_docs/swebench.pdf', 'section_id': 'A: BENCHMARK DETAILS', 'sub_section_id': 'A.3: Execution-Based Validation'}
{'page_num': 19, 'paper_path': 'iclr_docs/swebench.pdf', 'section_id': 'A: BENCHMARK DETAILS', 'sub_section_id': 'A.3: Execution-Based Validation'}
{'page_num': 20, 'paper_path': 'iclr_docs/swebench.pdf', 'section_id': 'A: BENCHMARK DETAILS', 'sub_section_id': 'A.5: Evaluation Test Set Characterization'}
{'page_num': 21, 'paper_path': 'iclr_docs/swebench.pdf', 'section_id': 'A: BENCHMARK DETAILS', 'sub_section_id': 'A.5: Ev

In [ ]:
response = query_engine.query(
    "Give me details of all additional experimental results in the Metra paper"
)
print(str(response))

>> Identifying the right sections to retrieve
>> Retrieving section: F: ADDITIONAL RESULTS
>> Retrieving section: 5: EXPERIMENTS
>> Retrieving section: F: ADDITIONAL RESULTS
The additional experimental results in the METRA paper include several key findings:

1. **Full Qualitative Results**: METRA discovers diverse locomotion behaviors across different environments, including state-based Ant and HalfCheetah, and pixel-based Quadruped and Humanoid. The results are consistent across multiple random seeds, indicating robustness in behavior discovery.

2. **Latent Space Visualization**: METRA effectively captures the most temporally spread-out dimensions in the state space, such as x-y coordinates, in its latent space. This is demonstrated in both state-based and pixel-based environments, with higher-dimensional latent spaces capturing more diverse behaviors.

3. **Ablation Study of Latent Space Sizes**: The study shows that increasing the size of the latent space generally enhances the di